# Data Cleaning

In [1]:
# import the relevent libraries
import pandas as pd
import numpy as np
import glob


In [2]:
# # Load the data

# dfs = []

# # Get all html files
# html_files = glob.glob("../data/raw/*.html")

# for f in html_files:
#     tables = pd.read_html(f, encoding='utf-8')
#     dfs.append(tables[0]) 

# merged_df = pd.concat(dfs, ignore_index=True)


In [3]:
# # Uniform column names
# merged_df.columns = merged_df.columns.str.strip().str.lower().str.replace(' ', '_')

# # Save to interim folder
# merged_df.to_csv("../data/interim/fm24_merged_raw.csv", index=False)

In [18]:
merged_df = pd.read_csv("../data/interim/fm24_merged_raw.csv")

In [19]:
# Drop duplicates
merged_df = merged_df.drop_duplicates()

In [20]:
print(" | ".join(merged_df.columns))

name | position | best_pos | age | nat | height | weight | personality | club | transfer_value | left_foot | right_foot | av_rat | gls | pens_s | pen/r | shot/90 | shot_% | sht/90 | xg | asts/90 | ps_c/90 | pas_% | k_ps/90 | ccc | cr_a | cr_c/a | tck/90 | tck_r | k_tck | int/90 | clear | fls | aer_a/90 | hdrs_w/90 | hdr_% | gl_mst | drb/90 | fa | dist/90 | yel | red | off | apps | mins | shots | ast | pens | mins/gm | sht | hdrs_a | cr_c | distance | drb | hdrs | itc | k_pas | pas_a | ps_c | tck_a | tck_c | expires | salary | preferred_foot | acc | aer | agg | agi | ant | bal | bra | cmd | com | cmp | cnt | cor | cro | dec | det | dri | ecc | fin | fir | fla | fre | han | hea | jum | kic | ldr | lon | l_th | mar | nat.1 | otb | 1v1 | pac | pas | pen | pos | pun | ref | tro | sta | str | tck | tea | tec | thr | vis | wor | k_hdrs/90 | blk/90 | xg/shot | sprints/90 | poss_won/90 | poss_lost/90 | pres_a/90 | pres_c/90 | pr_passes/90 | xa/90 | blk | xgp | xsv_% | conv_% | op-kp/90 | svt | 

In [21]:
cols_to_drop = ['personality','media_description','media_handling','inf','rec']
merged_df.drop(columns=cols_to_drop, inplace=True)

cols_to_rename = {'nat':'nation', 'nat.1':'nat', 'salary':'wages' }
merged_df.rename(columns=cols_to_rename, inplace=True)

In [22]:
# Replace only exact '-' with NaN
merged_df.replace('-', np.nan, inplace=True)

In [23]:
# Filter players who played for more than 600 minutes in all competitions
merged_df['mins'] = pd.to_numeric(merged_df['mins'], errors='coerce')
merged_df = merged_df[merged_df['mins'] >= 600]

In [24]:
# check how many and which columns have nulls
merged_df.isnull().sum()[merged_df.isnull().sum()>0]

left_foot                 3
right_foot                3
pen/r                   766
shot/90                  88
shot_%                   88
                       ... 
op-crs_a/90              93
op-crs_c/90              93
shts_blckd/90            93
shots_outside_box/90     93
saves/90                795
Length: 90, dtype: int64

In [25]:
# Remove units from height, weight and wages
merged_df['height'] = merged_df['height'].str.extract(r'(\d+)')
merged_df['weight'] = merged_df['weight'].str.extract(r'(\d+)')
merged_df['wages'] = merged_df['wages'].str.replace('[£, p/w]', '', regex=True)
merged_df[['height','weight','wages']] = merged_df[['height','weight','wages']].apply(pd.to_numeric, errors='coerce')

In [26]:
foot_values = {
    'Very Strong' : 10,
    'Fairly Strong' : 8,
    'Strong' : 6,
    'Reasonable' : 4,
    'Weak': 2,
    'Very Weak' : 0
}

merged_df[['left_foot', 'right_foot']] = merged_df[['left_foot', 'right_foot']].fillna('Very Weak')

merged_df['lf_val'] = merged_df['left_foot'].apply(lambda x: foot_values[x])
merged_df['rf_val'] = merged_df['right_foot'].apply(lambda x: foot_values[x])

merged_df[['lf_val', 'rf_val']] =  merged_df[['lf_val', 'rf_val']].apply(pd.to_numeric, errors='coerce')

In [27]:
merged_df['feet'] = merged_df['lf_val'] + merged_df['rf_val']

In [28]:
merged_df['feet']

0       18
1       14
2       14
3       18
4       18
        ..
2360    14
2361    12
2362    12
2363    14
2364    12
Name: feet, Length: 888, dtype: int64

In [29]:
merged_df[merged_df['left_foot'] == 'Weak'][['left_foot', 'right_foot'] + [ 'lf_val', 'rf_val']]

,left_foot,right_foot,lf_val,rf_val
6,Weak,Very Strong,2,10
26,Weak,Very Strong,2,10
72,Weak,Very Strong,2,10
74,Weak,Very Strong,2,10
80,Weak,Very Strong,2,10
...,...,...,...,...
2349,Weak,Very Strong,2,10
2358,Weak,Very Strong,2,10
2361,Weak,Very Strong,2,10
2362,Weak,Very Strong,2,10


In [ ]:
import re

def parse_value_range(value_str):
    """
    Parse Football Manager transfer value strings into min/max numeric values (GBP).
    
    Examples:
    ----------
    £54M - £67M → (54000000.0, 67000000.0)
    £45M → (45000000.0, 45000000.0)
    Not for Sale → (500000000.0, 500000000.0)
    Unknown → (NaN, NaN)
    """
    if not isinstance(value_str, str) or value_str.strip() == (""):
        return (np.nan, np.nan)
    
    value_str = value_str.strip()

    # Handle 'Not for sale'
    if value_str.lower() == 'not for sale':
        return (500_000_000.0, 500_000_000.0)
    
    # Handle Unknown or Loan or Free Transfer
    if value_str.lower() == 'unknown':
        return (np.nan, np.nan)
    
    # Handle ranges like "£54M - £67M"
    if "-" in value_str:
        parts = [p.strip() for p in value_str.split('-')]
        values = [parse_value_range(p)[0] for p in parts]
        if len(values) == 2:
            return (min(values), max(values))
        else:
            return (values[0], values[0]) if values else (np.nan, np.nan)
    
    # Extract numeric value with suffix
    match = re.search(r"£([\d\.]+)([KMB]?)", value_str)
    if not match:
        return (np.nan, np.nan)
    
    num = float(match.group(1))
    suffix = match.group(2)
    multiplier = {"K": 1_000, "M": 1_000_000, "B": 1_000_000_000, "": 1}

    value = num * multiplier.get(suffix, 1)
    
    return (value, value)

In [31]:
# spliting the transfer value into min and max value
value_ranges = merged_df['transfer_value'].apply(parse_value_range)

merged_df['min_value'] = value_ranges.apply(lambda x: x[0])
merged_df['max_value'] = value_ranges.apply(lambda x: x[1])

In [32]:
# Remove ' %' in the percentage columns
cols_with_percent = [col for col in merged_df.columns if merged_df[col].astype(str).str.contains('%').any()]
for col in cols_with_percent:
    merged_df[col] = merged_df[col].str.rstrip('%').astype(float) / 100

In [33]:
# Remove 'km' from distance column
merged_df['dist/90'] = merged_df['dist/90'].str.replace('[km]', '', regex=True)
merged_df['distance'] = merged_df['distance'].str.replace('[km]', '', regex=True)

In [36]:
# For the attribute columns, we would entries '3-7' so what i can do is get the 
def get_att_value(value):
    if isinstance(value, str) and '-' in value:
        low, high = map(float, value.split('-'))
        return (low + high) / 2
    return value

# Apply this function to the attribute columns
attributes = [
    "acc", "aer", "agg", "agi", "ant", "bal", "bra", "cmd", "com", "cmp", "cnt", "cor", "cro",
    "dec", "det", "dri", "ecc", "feet", "fin", "fir", "fla", "fre", "han", "hea", "jum", "kic", "ldr",
    "lon", "l_th", "mar", "nat", "otb", "1v1", "pac", "pas", "pen", "pos", "pun", "ref", "tro",
    "sta", "str", "tck", "tea", "tec", "thr", "vis", "wor"
]
for col in attributes:
    merged_df[col] = merged_df[col].apply(get_att_value)
    merged_df[col] = pd.to_numeric(merged_df[col], errors='coerce')
    merged_df[col] = merged_df.groupby('best_pos')[col].transform(lambda x: x.fillna(round(x.mean(),2)))

In [37]:
demograph_cols = ['name', 'position', 'best_pos', 'age', 'nation', 'height', 'weight', 'club', 'transfer_value','min_value', 'max_value', 'expires', 'wages', 'left_foot', 'right_foot', 'lf_val', 'rf_val', 'preferred_foot', 'division', 'style']

In [38]:
performance_stats = ['av_rat', 'gls', 'pens_s', 'pen/r', 'shot/90', 'shot_%', 'sht/90', 'xg', 'asts/90', 'ps_c/90', 'pas_%', 'k_ps/90', 'ccc', 'cr_a', 'cr_c/a', 'tck/90', 'tck_r', 'k_tck', 'int/90', 'clear', 'fls', 'aer_a/90', 'hdrs_w/90', 'hdr_%', 'gl_mst', 'drb/90', 'fa', 'dist/90', 'yel', 'red', 'off', 'mins', 'shots', 'ast', 'pens', 'mins/gm', 'sht', 'hdrs_a', 'cr_c', 'distance', 'drb', 'hdrs', 'itc', 'k_pas', 'pas_a', 'ps_c', 'tck_a', 'tck_c', 'k_hdrs/90', 'blk/90', 'xg/shot', 'sprints/90', 'poss_won/90', 'poss_lost/90', 'pres_a/90', 'pres_c/90', 'pr_passes/90', 'xa/90', 'blk', 'xgp', 'xsv_%', 'conv_%', 'op-kp/90', 'svt', 'xgp/90', 'op-kp', 'sv_%', 'svh', 'svp', 'all/90', 'cln/90', 'np-xg/90', 'pens_saved_ratio', 'np-xg', 'fk_shots', 'ch_c/90', 'clr/90', 'crs_a/90', 'cr_c/90', 'goals_outside_box', 'hdrs_l/90', 'k_tck/90', 'op-cr_%', 'op-crs_a/90', 'op-crs_c/90', 'ps_a/90', 'shts_blckd/90', 'shots_outside_box/90', 'xg/90', 'saves/90']

In [39]:
merged_df[performance_stats] = merged_df[performance_stats].fillna('0')
merged_df[performance_stats] = merged_df[performance_stats].apply(pd.to_numeric, errors='coerce')

In [40]:
merged_df['expires'] = pd.to_datetime(merged_df['expires'], format='%m/%d/%Y', errors='coerce')

In [41]:
# check how many and which columns have nulls
merged_df.isnull().sum()[merged_df.isnull().sum()>0]

min_value    3
max_value    3
dtype: int64

In [42]:
merged_df['apps']

0        40 (2)
1        35 (2)
2        38 (6)
3        32 (4)
4            36
         ...   
2360      6 (7)
2361         30
2362         39
2363    21 (14)
2364         36
Name: apps, Length: 888, dtype: object

In [43]:
cleaned_df = merged_df[demograph_cols + attributes + performance_stats]

# Save cleaned dataset
cleaned_df.to_csv("../data/processed/players_2024_clean.csv", index=False)

In [44]:
cleaned_df.sample(10)

,name,position,best_pos,age,nation,height,weight,club,transfer_value,min_value,...,hdrs_l/90,k_tck/90,op-cr_%,op-crs_a/90,op-crs_c/90,ps_a/90,shts_blckd/90,shots_outside_box/90,xg/90,saves/90
578,Marten de Roon,"DM, M (C)",DM,33.0,NED,185,76,Atalanta,Not for Sale,500000000.0,...,1.47,0.13,0.22,0.48,0.11,68.45,0.40,0.61,0.08,0
1037,Andrew Omobamidele,D (C),D (C),21.0,IRL,190,85,Nottingham Forest,£16M,16000000.0,...,1.58,0.44,0.00,0.00,0.00,45.51,0.63,0.06,0.06,0
882,Gabriel,D (C),D (C),26.0,BRA,190,78,Arsenal,£173M - £207M,173000000.0,...,1.39,0.34,0.00,0.10,0.00,80.28,0.10,0.00,0.04,0
345,José Giménez,"D (C), DM",D (C),29.0,URU,185,77,Atlético Madrid,£49M - £71M,49000000.0,...,2.39,0.13,0.08,0.33,0.03,75.89,0.36,0.08,0.05,0
382,Thibaut Courtois,GK,GK,32.0,BEL,200,96,Real Madrid,Not for Sale,500000000.0,...,0.00,0.00,0.00,0.00,0.00,20.51,0.00,0.00,0.00,2
565,Jasmin Kurtič,"DM, M/AM (C)",M (C),35.0,SVN,186,77,Monza,£10K - £35K,10000.0,...,1.93,0.12,0.13,0.75,0.12,39.92,0.25,0.81,0.06,0
43,Yussuf Poulsen,ST (C),ST (C),29.0,DEN,192,84,RB Leipzig,£32M - £40M,32000000.0,...,3.03,0.10,0.18,2.30,0.42,26.90,0.00,1.36,0.79,0
40,Jakub Kamiński,M/AM (RL),AM (L),21.0,POL,179,68,VfL Wolfsburg,£42M - £66M,42000000.0,...,2.42,0.03,0.27,3.05,0.83,44.48,0.00,0.48,0.20,0
346,Arsen Zakharyan,"M (C), AM (RLC)",AM (C),21.0,RUS,183,72,Real Sociedad,£12M - £14.5M,12000000.0,...,1.37,0.00,0.20,1.14,0.31,48.74,0.08,0.76,0.16,0
413,Pau López,GK,GK,29.0,ESP,189,77,Girona,£16M - £23M,16000000.0,...,0.00,0.00,0.00,0.00,0.00,37.25,0.00,0.00,0.00,3
